In [ ]:
import os
import pandas as pd
from google.cloud import storage
storage_client = storage.Client()
import google.cloud

from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import *

# TRANSFORMAÇÃO NETFLIX

In [ ]:
def lista_objetos(bucket_name):
    """Lista todos os objetos da pasta"""
    blobs = storage_client.list_blobs(bucket_name)

    for blob in blobs:
        print(blob.name)

In [ ]:
lista_objetos('datalake-5gflix')

erros.csv
landing/
landing/amazon_reviews_us_Digital_Video_Download_v1_00.tsv.gz
landing/amazon_reviews_us_Video_DVD_v1_00.tsv.gz
landing/amazon_reviews_us_Video_v1_00.tsv.gz
landing/nf_prize_dataset.tar.gz
processing/
processing/amazon_reviews_us_Digital_Video_Download_v1_00.tsv
processing/amazon_reviews_us_Video_DVD_v1_00.tsv
processing/amazon_reviews_us_Video_v1_00.tsv
processing/nf_prize_dataset/README
processing/nf_prize_dataset/combined_data.csv
processing/nf_prize_dataset/movie_titles.csv
processing/nf_prize_dataset/probe.txt
processing/nf_prize_dataset/qualifying.txt
temp/
trusted/
trusted/df_netflix_parquet.parquet/
trusted/df_netflix_parquet.parquet/_SUCCESS
trusted/df_netflix_parquet.parquet/part-00000-b2e7d38b-4231-49db-9c9d-bfb2efdce7d0-c000.snappy.parquet
trusted/df_netflix_parquet.parquet/part-00001-b2e7d38b-4231-49db-9c9d-bfb2efdce7d0-c000.snappy.parquet
trusted/df_netflix_parquet.parquet/part-00002-b2e7d38b-4231-49db-9c9d-bfb2efdce7d0-c000.snappy.parquet
trusted/df_net

In [ ]:
bucket=storage_client.bucket('datalake-5gflix')
blob = bucket.blob('processing/nf_prize_dataset/combined_data_4.txt')
blob.download_to_filename('/combined_data_4.txt')
print('Download concluido')

In [ ]:
# Realizando a leitura dos quatro datasets para união em um único arquivo csv.
if not os.path.isfile('/combined_data.csv'):
    
    data = open('combined_data.csv', mode='w')
    
    armazen = list()
    arquivos = [
        '/combined_data_1.txt',
        '/combined_data_2.txt',
        '/combined_data_3.txt',
        '/combined_data_4.txt'   
    ]
    for arquivo in arquivos:
        print(f"Lendo {arquivo}\n")
        with open(arquivo) as a:
            for line in a: 
                line = line.strip()
                if line.endswith(':'):   #insere o id do filme em todas as linhas até que o próximo filme apareça
                    
                    id_filme = line.replace(':', '')
                else:
                    armazen = [x for x in line.split(',')]
                    armazen.insert(0, id_filme)
                    data.write(','.join(armazen))
                    data.write('\n')
    data.close()
print('Processo concluído')

Lendo /combined_data_1.txt

Lendo /combined_data_2.txt

Lendo /combined_data_3.txt

Lendo /combined_data_4.txt

Processo concluído


In [ ]:
# Envio do dataset para camada processing

def upload_dataset(bucket_name, source_file_name, destination_blob_name):
    '''Upload dos aquivos no datalake'''
   
    # bucket_name = "nome-da-bucket"
    # source_file_name =  caminho do arquivo a ser enviado
    # destination_blob_name = "nome-objeto-bucket"

    
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        f"Arquivo {source_file_name} enviado para {destination_blob_name}."
    )

In [ ]:
upload_dataset('datalake-5gflix','/combined_data.csv','processing/nf_prize_dataset/combined_data.csv')  

Arquivo /combined_data.csv enviado para processing/nf_prize_dataset/combined_data.csv.


In [ ]:
# Definindo o schema
schema = (
    StructType([
        StructField("id_movie",IntegerType(), True),
        StructField("id_user" ,IntegerType(), True),
        StructField("rating", IntegerType(), True),
        StructField("date", DateType(), True),
    ])
)    

In [ ]:
#Leitura do dataset netflix com as avaliações
path='gs://datalake-5gflix/processing/nf_prize_dataset/combined_data.csv'

df = (
    spark.read.format("csv")
    .option("delimiter", ",")
    .option("header","false")
    .option("inferschema","true")
    .load(path,schema=schema)
)

In [ ]:
df.printSchema()

root
 |-- id_movie: integer (nullable = true)
 |-- id_user: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- date: date (nullable = true)



In [ ]:
df.count()

100480507

In [ ]:
df.show(10)

+--------+-------+------+----------+
|id_movie|id_user|rating|      date|
+--------+-------+------+----------+
|       1|1488844|     3|2005-09-06|
|       1| 822109|     5|2005-05-13|
|       1| 885013|     4|2005-10-19|
|       1|  30878|     4|2005-12-26|
|       1| 823519|     3|2004-05-03|
|       1| 893988|     3|2005-11-17|
|       1| 124105|     4|2004-08-05|
|       1|1248029|     3|2004-04-22|
|       1|1842128|     4|2004-05-09|
|       1|2238063|     3|2005-05-11|
+--------+-------+------+----------+
only showing top 10 rows



In [ ]:
df.tail(10)

[Row(_c0=17770, _c1=834323, _c2=2, _c3='2005-10-04'),
 Row(_c0=17770, _c1=516110, _c2=5, _c3='2004-08-26'),
 Row(_c0=17770, _c1=365996, _c2=3, _c3='2003-11-10'),
 Row(_c0=17770, _c1=986348, _c2=4, _c3='2004-08-12'),
 Row(_c0=17770, _c1=311124, _c2=3, _c3='2005-09-29'),
 Row(_c0=17770, _c1=1790158, _c2=4, _c3='2005-11-01'),
 Row(_c0=17770, _c1=1608708, _c2=3, _c3='2005-07-19'),
 Row(_c0=17770, _c1=234275, _c2=1, _c3='2004-08-07'),
 Row(_c0=17770, _c1=255278, _c2=4, _c3='2004-05-28'),
 Row(_c0=17770, _c1=453585, _c2=2, _c3='2005-03-10')]

In [ ]:
# Dropando as colunas irrelevantes para o estudo
df=df.drop(F.col('id_user'))
df=df.drop(F.col('date'))

In [ ]:
df.show()

+--------+------+
|id_movie|rating|
+--------+------+
|       1|     3|
|       1|     5|
|       1|     4|
|       1|     4|
|       1|     3|
|       1|     3|
|       1|     4|
|       1|     3|
|       1|     4|
|       1|     3|
|       1|     4|
|       1|     5|
|       1|     3|
|       1|     3|
|       1|     4|
|       1|     4|
|       1|     4|
|       1|     3|
|       1|     4|
|       1|     5|
+--------+------+
only showing top 20 rows



In [ ]:
# Calculando a média de avaliação dos filmes, utilizando o calculo para média ponderada.

In [ ]:
# média de votos em todo conjunto de dados
df.agg(F.round(F.avg("rating"),1).alias("rating")).show()

+------+
|rating|
+------+
|   3.6|
+------+



In [ ]:
# Obtendo a informação da média de avaliação de cada filme e a quantidade de avaliações recebidas.

df2 = ( df.groupBy(F.col("id_movie"))
               .agg(F.round(F.avg("rating"),1).alias("rating"),
                F.count("rating").alias('qnt_rating'))

)

In [ ]:
df2.orderBy(F.col('id_movie').asc()).show(5)

+--------+------+----------+
|id_movie|rating|qnt_rating|
+--------+------+----------+
|       1|   3.7|       547|
|       2|   3.6|       145|
|       3|   3.6|      2012|
|       4|   2.7|       142|
|       5|   3.9|      1140|
+--------+------+----------+
only showing top 5 rows



In [ ]:
# pegamos a quantidade de avaliações de cada filme,dividimos pela quantidade de avaliações adcionada a 1000.
# multiplicamos pela soma da nota média do filme + 1000
# dividimos pela quantidade de avaliações + 1000 e multiplicamos pela média de avalização geral "3.6"
df = df2.withColumn('Rating',F.lit(F.col('qnt_rating')/(F.col('qnt_rating')+1000)*(F.col('rating')+1000/(F.col('qnt_rating')+1000)*3.6)))

In [ ]:
df.count()

17770

In [ ]:
#Schema para o dataset netflix com os nomes de filmes
schema = (
    StructType([
        StructField("id_movie_",IntegerType(), True),
        StructField("launch" ,IntegerType(), True),
        StructField("title", StringType(), True)        
    ])
)    

In [ ]:
#Leitura do dataset netflix com os nomes de filmes
path='gs://datalake-5gflix/processing/nf_prize_dataset/movie_titles.csv'

movie = (
    spark.read.format("csv")
    .option("delimiter", ",")
    .option("header","false")
    .option("inferschema","true")
    .load(path,schema=schema)
)

In [ ]:
movie.show(10,truncate=False)

+---------+------+----------------------------+
|id_movie_|launch|title                       |
+---------+------+----------------------------+
|1        |2003  |Dinosaur Planet             |
|2        |2004  |Isle of Man TT 2004 Review  |
|3        |1997  |Character                   |
|4        |1994  |Paula Abdul's Get Up & Dance|
|5        |2004  |The Rise and Fall of ECW    |
|6        |1997  |Sick                        |
|7        |1992  |8 Man                       |
|8        |2004  |What the #$*! Do We Know!?  |
|9        |1991  |Class of Nuke 'Em High 2    |
|10       |2001  |Fighter                     |
+---------+------+----------------------------+
only showing top 10 rows



In [ ]:
movie.count()

17770

In [ ]:
# Realiza a junção dos 2 datasets netflix
df_netflix = df.join(movie, df.id_movie == movie.id_movie_,how='left')

In [ ]:
df_netflix.show()

+--------+------------------+----------+---------+------+--------------------+
|id_movie|            Rating|qnt_rating|id_movie_|launch|               title|
+--------+------------------+----------+---------+------+--------------------+
|    3997|0.8102450641377344|       159|     3997|  1983|   Of Unknown Origin|
|    4101|0.4433272967416022|        84|     4101|  1983|          Hysterical|
|    4519| 2.909123769229049|      2806|     4519|  2003|      Out for a Kill|
|    4818|0.6596624652545954|       121|     4818|  1973|       Le Magnifique|
|    4900|1.0771237707660328|       281|     4900|  2002|Let's Dance Salsa...|
|    4158|0.5994462562385338|       117|     4158|  1986|Burial Ground: Th...|
|    4161| 3.076215207774021|      1799|     4161|  1997|        I Think I Do|
|    4190|0.7323215422842088|       132|     4190|  2003|Hugh Hefner: Amer...|
|    4219|3.8496351496925594|     10942|     4219|  1995| The Tuskegee Airmen|
|    4929|0.7037013610869529|       119|     4929|  

In [ ]:
df_netflix = df_netflix.drop(F.col('id_movie_'))

In [ ]:
# Removendo os títulos duplicados
df_netflix = df_netflix.dropDuplicates(['title'])

In [ ]:
# Criando uma nova coluna com o nome da empresa
df_netflix=df_netflix.withColumn('company',F.lit('netflix'))

In [ ]:
df_netflix.show(5)

+--------+------------------+----------+------+--------------------+-------+
|id_movie|            Rating|qnt_rating|launch|               title|company|
+--------+------------------+----------+------+--------------------+-------+
|     148| 3.306242452575187|     22194|  2001|      Sweet November|Netflix|
|     463|3.6144379464105456|      2286|  1962|The Twilight Zone...|Netflix|
|     471| 3.890082644628099|      4500|  1931|         City Lights|Netflix|
|     496|1.2587208444655706|       235|  2004|Primus: Hallucino...|Netflix|
|     833| 3.469729456247705|      8378|  1996|        White Squall|Netflix|
+--------+------------------+----------+------+--------------------+-------+
only showing top 5 rows



### Limpando a coluna titulos

In [ ]:
# Removendo caracters invalidos
df_netlix = df_netflix.withColumn('title', regexp_replace('title', '([^\s\w_]|_)+', ''))

# Removendo espaços duplos
df_netflix = df_netflix.withColumn('title', regexp_replace('title', '  ', ' '))

# Removendo espaços do inicio e final
df_netflix = df_netflix.withColumn('title',trim(F.col('title')))

# Alterando os titulos para letras mínusculas
df_netflix = df_netflix.withColumn("title", lower(F.col("title")))

In [ ]:
# Converte para formato parquet e armazena na camada trusted
df_netflix.write.format("parquet")\
.mode("overwrite")\
.option('header', 'true') \
.save("gs://datalake-5gflix/trusted/df_netflix_parquet.parquet")

In [ ]:
dfnetp = spark.read.format("parquet")\
.option('header','true')\
.load("gs://datalake-5gflix/trusted/df_netflix_parquet.parquet")

In [ ]:
dfnetp.printSchema()

root
 |-- id_movie: integer (nullable = true)
 |-- Rating: double (nullable = true)
 |-- qnt_rating: long (nullable = true)
 |-- launch: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- company: string (nullable = true)



In [ ]:
dfnetp.show(50)

+--------+-------------------+----------+------+--------------------+-------+
|id_movie|             Rating|qnt_rating|launch|               title|company|
+--------+-------------------+----------+------+--------------------+-------+
|      98| 0.8151852141906178|       130|  1965|The Battle of Alg...|netflix|
|     195| 0.9894728408908829|       185|  2004|     Chasing Freedom|netflix|
|     256|  3.394616153576402|     12264|  2000|Ghost Dog: The Wa...|netflix|
|     678| 1.3435941306607024|       282|  1965|   Loves of a Blonde|netflix|
|     714|0.39170547670706607|        76|  1988|        Whisper Kill|netflix|
|     911|  1.869927689051522|       539|  1939|         Buck Rogers|netflix|
|     933| 1.4264795767721163|       263|  2005|Backyardigans: It...|netflix|
|    1042| 2.2601670531389235|       722|  1996|    Forgotten Silver|netflix|
|     287| 1.2960000000000003|       250|  1994|Luther Vandross: ...|netflix|
|     345| 3.8689561016593146|      4175|  1998|Star Trek: Voyag

In [ ]:
dfnetp.count()

17297

In [ ]:
# conta a quantidade de linhas nulas
print ("Valores nulos coluna title: {0}".format(dfnetp.filter("title is NULL").count()))

Valores nulos coluna title: 0


In [ ]:
# organizando as colunas para junção dos dataframes netflix + amazon
dfnetp = dfnetp.select(F.col('id_movie'),F.col('title'),F.col('Rating').alias('rating'),F.col('launch'),F.col('company'))

# TRANSFORMAÇÃO AMAZON

In [ ]:
path='gs://datalake-5gflix/processing/*.tsv'

df_amz = (
    spark.read.format("csv")
    .option("delimiter", "\t")
    .option("encoding", "UTF-8")
    .option("header","True")
    .option("inferschema","true")
    .load(path)
)

In [ ]:
df_amz.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [ ]:
# Selecionando apenas as colunas relevantes e criando o dataframe
df_amz=df_amz.select(F.col('product_id'),F.col('product_title'),F.col('star_rating'),F.col('review_date'))

In [ ]:
df_amz.count()

9506891

In [ ]:
df_amz.orderBy(F.col('review_date').asc()).show()

+----------+--------------------+-----------+-----------+
|product_id|       product_title|star_rating|review_date|
+----------+--------------------+-----------+-----------+
|B002RTFHUU|      Angela's Ashes|          5|       null|
|B00FRAV1BU|American Horror S...|          5|       null|
|B000Q3DWD8|Ron White: You Ca...|          5|       null|
|B00EEN66SY|      The Bling Ring|          3|       null|
|B00GBD1518|Gas House Kids in...|          5|       null|
|B000QYBF22|        Black Monday|          5|       null|
|B00CDZFRAI|Alpha House Season 1|          4|       null|
|B004W1GCNC|Upstairs Downstai...|          5|       null|
|B00E9PXI3S|             Passion|          1|       null|
|B00271FP00|         Black Sheep|          4|       null|
|B00F88H5K2|Behind the Candel...|          2|       null|
|B00332MY6K|   White Heat (1949)|          5|       null|
|B009NXV51S|       Almost Famous|          5|       null|
|B00ES3NOBC|Execution of Raym...|          3|       null|
|B008Y7N7JW|  

In [ ]:
# conta a quantidade de linhas nulas
print ("Valores nulos coluna Product_title: {0}".format(df_amz.filter("product_title is NULL").count()))

Valores nulos coluna Product_title: 0


### Limpando a coluna titulos

In [ ]:

# Removendo caracters invalidos
df_amz = df_amz.withColumn('product_title', regexp_replace('product_title', '([^\s\w_]|_)+', ''))

# Removendo espaços duplos
df_amz = df_amz.withColumn('product_title', regexp_replace('product_title', '  ', ' '))

# Removendo espaços do inicio e final
df_amz = df_amz.withColumn('product_title',trim(F.col('product_title')))

# Alterando os titulos para letras mínusculas
df_amz = df_amz.withColumn("product_title", lower(F.col("product_title")))


In [ ]:
# Extrai ano da data

'''id_movie title rating lauch company'''

df_amz=df_amz.select(year(F.col("review_date")).alias("year"),F.col('product_id').alias('movie_id'),F.col('product_title').alias('title'),F.col('star_rating').alias('rating'))

In [ ]:
# Criando a coluna company amazon
df_amz=df_amz.withColumn('company',F.lit('amazon'))

In [ ]:
# Removendo os títulos duplicados
df_amz = df_amz.dropDuplicates(['title'])

In [ ]:
# organizando as colunas para junção dos dataframes netflix + amazon
df_amz = df_amz.select(F.col('movie_id').alias('id_movie'),F.col('title'),F.col('rating'),F.col('year').alias('launch'),F.col('company'))

In [ ]:
df_amz.show()

+----------+--------------------+------+-----+-------+
|  id_movie|               title|rating|lauch|company|
+----------+--------------------+------+-----+-------+
|B00AYB1482|enlightened season 1|     5| 2015| Amazon|
|B00KQD28OM|             vicious|     5| 2015| Amazon|
|B01489L5LQ|         after words|     4| 2015| Amazon|
|B008LOVIIK|masterpiece inspe...|     5| 2015| Amazon|
|B0094LZMT0|   on the waterfront|     5| 2015| Amazon|
|B0112OSOQE|rick and morty se...|     5| 2015| Amazon|
|B000NPE5SA|      africa screams|     4| 2015| Amazon|
|B00XWV4QXG|  entourage season 7|     3| 2015| Amazon|
|B00X8UKOUK|catastrophe season 1|     2| 2015| Amazon|
|B00OOKXTFU|the worricker tri...|     3| 2015| Amazon|
|B00URDJ600|mr selfridge seas...|     5| 2015| Amazon|
|B009JUZ5I4|  five branded women|     5| 2015| Amazon|
|B00PSLQYWE|downton abbey sea...|     4| 2015| Amazon|
|B00LI1COYA|     extant season 1|     5| 2015| Amazon|
|B01347T9O8|            casanova|     1| 2015| Amazon|
|B00EC7VQX

In [ ]:
# Converte para formato parquet e armazena na camada trusted
df_amz.write.format("parquet")\
.mode("overwrite")\
.option('header', 'true') \
.save("gs://datalake-5gflix/trusted/df_amz_parquet.parquet")

In [ ]:
df_amz = spark.read.format("parquet")\
.option('header','true')\
.load("gs://datalake-5gflix/trusted/df_amz_parquet.parquet")

In [ ]:
df_amz.printSchema()

root
 |-- id_movie: string (nullable = true)
 |-- title: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- launch: integer (nullable = true)
 |-- company: string (nullable = true)



In [ ]:
df_amz.show()

+----------+--------------------+------+-----+-------+
|  id_movie|               title|rating|lauch|company|
+----------+--------------------+------+-----+-------+
|B007I1Q4MM|the magic school ...|     5| 2015| Amazon|
|B005BUA1GC|melissa joey seas...|     5| 2015| Amazon|
|B0000DZ3EG|      follow me boys|     4| 2015| Amazon|
|B005NQ5LLY|d grayman season ...|     5| 2015| Amazon|
|B00062IYZK|popeye the sailor v2|     5| 2015| Amazon|
|B00Q7WBGHG|      into the woods|     5| 2015| Amazon|
|B00008977A|    man of the house|     5| 2015| Amazon|
|B005TBQS3I|dead poets societ...|     5| 2015| Amazon|
|B002C2KAS6|       ncis season 6|     5| 2015| Amazon|
|B00HRYH7DA|jesus film 35th a...|     5| 2015| Amazon|
|B005FQ2HDC|   black moon rising|     5| 2015| Amazon|
|B00MRAJGDI|the expendables 3...|     5| 2015| Amazon|
|B00R3DO58K|the hobbit the ba...|     5| 2015| Amazon|
|B00JHH1ZN8|rodgers hammerste...|     5| 2015| Amazon|
|B00RGQ4674|outlander season ...|     5| 2015| Amazon|
|B002U0KHN

In [ ]:
# União dos 2 dataframes.
df_final = dfnetp.unionByName(df_amz)

In [ ]:
df_final.show()

+--------+--------------------+-------------------+------+-------+
|id_movie|               title|             rating|launch|company|
+--------+--------------------+-------------------+------+-------+
|     882|gloria estefan ev...|0.46435129031266126|  1996|netflix|
|    3579|    2 fast 2 furious|  3.305520298750795|  2003|netflix|
|    7940|            robotech|  3.237631823352444|  1985|netflix|
|    8655|               blood| 0.9123321801638006|  1998|netflix|
|   12054|         childs play| 3.0247819965165474|  1988|netflix|
|    5047|max keebles big move| 3.1529620181405895|  2001|netflix|
|   11404|mtv the real worl...| 2.2982157773397036|  2002|netflix|
|   16273|        into the sun|   2.81504902742347|  2004|netflix|
|     416|            elephant|  3.024989092569995|  2003|netflix|
|    3060|midnight street r...|  1.182282996432818|  2002|netflix|
|    6993|         varians war| 2.0458658424222618|  2000|netflix|
|    7699|bratz the video s...| 2.3702376566374674|  2004|netf

In [ ]:
# Converte para formato parquet e armazena na camada trusted
df_final.write.format("parquet")\
.mode("overwrite")\
.option('header', 'true') \
.save("gs://datalake-5gflix/trusted/df_final_parquet.parquet")